## Creating a SQLite Database from our CSV Source Files

Using pandas and sqlite to create our .db file. We use os to prevent errors if our .db file and tables already exists with future, successive runs of the notebook. Matplotlib is required for a brief visualization at the conclusion.

In [80]:
import pandas as pd
import sqlite3
import os
import matplotlib as plt

We use pandas to load the csv into a dataframe and display the first few rows

In [81]:
# Load bites CSV to dataframe
bite_df = pd.read_csv('assets/Health_AnimalBites.csv')
print(bite_df.head())

             bite_date SpeciesIDDesc BreedIDDesc GenderIDDesc       color  \
0  1985-05-05 00:00:00           DOG         NaN       FEMALE  LIG. BROWN   
1  1986-02-12 00:00:00           DOG         NaN      UNKNOWN   BRO & BLA   
2  1987-05-07 00:00:00           DOG         NaN      UNKNOWN         NaN   
3  1988-10-02 00:00:00           DOG         NaN         MALE   BLA & BRO   
4  1989-08-29 00:00:00           DOG         NaN       FEMALE     BLK-WHT   

   vaccination_yrs     vaccination_date victim_zip AdvIssuedYNDesc  \
0              1.0  1985-06-20 00:00:00      40229              NO   
1              NaN                  NaN      40218              NO   
2              NaN                  NaN      40219              NO   
3              NaN                  NaN        NaN              NO   
4              NaN                  NaN        NaN              NO   

  WhereBittenIDDesc      quarantine_date DispositionIDDesc head_sent_date  \
0              BODY  1985-05-05 00:00:0

Sqlite creates a database and our first table named Bites

In [82]:
# Create lunacy_db.db and add Bites table
# if statement skips process if lunacy_db.db already exists
if not os.path.isfile('lunacy_db.db'):
    with sqlite3.connect('lunacy_db.db') as conn:
        bite_df.to_sql('Bites', conn)
else:
    print("Database file already exists. Skipping creation.")

Database file already exists. Skipping creation.


We do the same for our csv with moon phase data

In [83]:
# Load moon CSV to dataframe
moon_df = pd.read_csv('assets/moon_illumination_1800-2100.csv')
print(moon_df.head())

   Unnamed: 0                    date  illum_pct           phase
0           0  1800-01-01 12:00:00.00      36.00   first quarter
1           1  1800-01-02 12:00:00.00      45.58   first quarter
2           2  1800-01-03 12:00:00.00      55.14   first quarter
3           3  1800-01-04 12:00:00.00      64.41   first quarter
4           4  1800-01-05 12:00:00.00      73.12  waxing gibbous


This will become the table called Moon

In [84]:
# Add Moon table to lunacy_db.db
# This block prevents errors if db and table already exist
if not os.path.isfile('lunacy_db.db'):
    with sqlite3.connect('lunacy_db.db') as conn:
        bite_df.to_sql('Bites', conn)
        moon_df.to_sql('Moon', conn)
else:
    with sqlite3.connect('lunacy_db.db') as conn:
        cur = conn.cursor()
        cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Bites'")
        bites_exists = cur.fetchone() is not None
        cur.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='Moon'")
        moon_exists = cur.fetchone() is not None

    if not bites_exists:
        with sqlite3.connect('lunacy_db.db') as conn:
            bite_df.to_sql('Bites', conn)

    if not moon_exists:
        with sqlite3.connect('lunacy_db.db') as conn:
            moon_df.to_sql('Moon', conn)

This data has very few entries before 2010 and after 2017. We will select our data from between these times.

## Query our SQL data

First, our animal bite data.

In [85]:
# Connect to lunacy_db.db and query the potentially relevant bite data
conn = sqlite3.connect("lunacy_db.db")

bite_df = pd.read_sql("""SELECT 
                            bite_date, 
                            SpeciesIDDesc, 
                            BreedIDDesc, 
                            GenderIDDesc, 
                            color, 
                            victim_zip, 
                            WhereBittenIDDesc, 
                            ResultsIDDesc 
                        FROM 
                            bites b 
                        WHERE 
                            b.bite_date 
                        BETWEEN 
                            '2009-10-29' and '2017-09-08' 
                        ORDER BY 
                            bite_date;""", conn)

# Quick peek
print(bite_df.head())

             bite_date SpeciesIDDesc BreedIDDesc GenderIDDesc    color  \
0  2009-10-29 00:00:00           CAT        None       FEMALE     GRAY   
1  2009-12-02 00:00:00           DOG        None         MALE  TAN-BRN   
2  2009-12-11 00:00:00           DOG        None         MALE  BLK-BRN   
3  2009-12-21 00:00:00           DOG        None      UNKNOWN    BLACK   
4  2009-12-24 00:00:00           DOG        None         MALE  BRN-WHT   

  victim_zip WhereBittenIDDesc ResultsIDDesc  
0      40206              BODY       UNKNOWN  
1      40291              BODY       UNKNOWN  
2      40272              BODY       UNKNOWN  
3      40218              HEAD       UNKNOWN  
4      40165              HEAD       UNKNOWN  


Now, our lunar data.

In [86]:
# Connect to lunacy_db.db and query the moon phase data
moon_df = pd.read_sql("""SELECT 
                            date, 
                            illum_pct, 
                            phase 
                        FROM 
                            moon m 
                        WHERE 
                            m.date 
                        BETWEEN 
                            '2009-10-29' and '2017-09-08' 
                        ORDER BY 
                            date;""", conn)

# Quick peek (again)
print(moon_df.head())

                     date  illum_pct           phase
0  2009-10-29 12:00:00.00      80.90  waxing gibbous
1  2009-10-30 12:00:00.00      88.09  waxing gibbous
2  2009-10-31 12:00:00.00      93.90  waxing gibbous
3  2009-11-01 12:00:00.00      97.94            full
4  2009-11-02 12:00:00.00      99.88            full


We can now begin to see what we're working with.

In [87]:
print(bite_df.info())
print(moon_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8631 entries, 0 to 8630
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   bite_date          8631 non-null   object
 1   SpeciesIDDesc      8530 non-null   object
 2   BreedIDDesc        3696 non-null   object
 3   GenderIDDesc       6355 non-null   object
 4   color              6307 non-null   object
 5   victim_zip         6879 non-null   object
 6   WhereBittenIDDesc  8303 non-null   object
 7   ResultsIDDesc      1328 non-null   object
dtypes: object(8)
memory usage: 539.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871 entries, 0 to 2870
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       2871 non-null   object 
 1   illum_pct  2871 non-null   float64
 2   phase      2871 non-null   object 
dtypes: float64(1), object(2)
memory usage: 67.4+ KB
None


First, we'll convert our two date columns to datetime64, then drop the times as one is empty and the other is noon.

In [88]:
# Convert to datetime and check dtype
bite_df['bite_date'] = pd.to_datetime(bite_df['bite_date'], errors='coerce')
print(bite_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8631 entries, 0 to 8630
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   bite_date          8631 non-null   datetime64[ns]
 1   SpeciesIDDesc      8530 non-null   object        
 2   BreedIDDesc        3696 non-null   object        
 3   GenderIDDesc       6355 non-null   object        
 4   color              6307 non-null   object        
 5   victim_zip         6879 non-null   object        
 6   WhereBittenIDDesc  8303 non-null   object        
 7   ResultsIDDesc      1328 non-null   object        
dtypes: datetime64[ns](1), object(7)
memory usage: 539.6+ KB
None


In [89]:
# Convert object to datetime again with moon data
moon_df['date'] = pd.to_datetime(moon_df['date'])
print(moon_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871 entries, 0 to 2870
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       2871 non-null   datetime64[ns]
 1   illum_pct  2871 non-null   float64       
 2   phase      2871 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 67.4+ KB
None


We will need to join our dataframes using the date. Currently our bite dataframe possesses empty timestamps in the datetime (00:00:00.00) and our lunar dataframe records all times at exactly noon (12:00:00.00). We will convert the datetime column to just the date. In doing so it is converted back to an object. We then repeat the process to again set the column as datetime64.

In [90]:
# Convert 'date' to only hold date and not time
# This makes it an object again
# Re-convert to datetime for merge compatability
moon_df['date'] = moon_df['date'].dt.date
print(moon_df.info())
moon_df['date'] = pd.to_datetime(moon_df['date'])
print(moon_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871 entries, 0 to 2870
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       2871 non-null   object 
 1   illum_pct  2871 non-null   float64
 2   phase      2871 non-null   object 
dtypes: float64(1), object(2)
memory usage: 67.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2871 entries, 0 to 2870
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       2871 non-null   datetime64[ns]
 1   illum_pct  2871 non-null   float64       
 2   phase      2871 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 67.4+ KB
None


We now combine our two dataframes on the date.

In [91]:
# Merge dataframes on dates
df = pd.merge(bite_df, moon_df, left_on='bite_date', right_on='date')
print(df.head())

   bite_date SpeciesIDDesc BreedIDDesc GenderIDDesc    color victim_zip  \
0 2009-10-29           CAT        None       FEMALE     GRAY      40206   
1 2009-12-02           DOG        None         MALE  TAN-BRN      40291   
2 2009-12-11           DOG        None         MALE  BLK-BRN      40272   
3 2009-12-21           DOG        None      UNKNOWN    BLACK      40218   
4 2009-12-24           DOG        None         MALE  BRN-WHT      40165   

  WhereBittenIDDesc ResultsIDDesc       date  illum_pct            phase  
0              BODY       UNKNOWN 2009-10-29      80.90   waxing gibbous  
1              BODY       UNKNOWN 2009-12-02      99.96             full  
2              BODY       UNKNOWN 2009-12-11      24.10  waning crescent  
3              HEAD       UNKNOWN 2009-12-21      21.11  waxing crescent  
4              HEAD       UNKNOWN 2009-12-24      47.87    first quarter  


For the sake of having a copy on file instead of memory, I will now save our merged dataframe as a CSV file.

In [92]:
# Savesies
df.to_csv('assets/merged_df.csv')

## EDA

Now that our data is all in one dataframe, is is time for a bit of exploratory data analysis.

In [93]:
# Describe
print(df.describe(include='all', datetime_is_numeric=True))

                            bite_date SpeciesIDDesc BreedIDDesc GenderIDDesc  \
count                            8631          8530        3696         6355   
unique                            NaN             9         101            3   
top                               NaN           DOG    PIT BULL         MALE   
freq                              NaN          6883        1078         3769   
mean    2013-10-14 19:02:31.407716352           NaN         NaN          NaN   
min               2009-10-29 00:00:00           NaN         NaN          NaN   
25%               2011-11-14 00:00:00           NaN         NaN          NaN   
50%               2013-09-12 00:00:00           NaN         NaN          NaN   
75%               2015-08-18 12:00:00           NaN         NaN          NaN   
max               2017-09-07 00:00:00           NaN         NaN          NaN   
std                               NaN           NaN         NaN          NaN   

        color victim_zip WhereBittenIDD

Let's see what interesting information we can glean from this dataset.

In [94]:
# What kinds of dogs are out there biting the citizens of Louisville?
breed_total = df["BreedIDDesc"].value_counts()
print(breed_total.head())

# What types of animals appear on this list?
unique_species = df['SpeciesIDDesc'].unique()
print(unique_species)

# How many bites from these different species?
species_total = df["SpeciesIDDesc"].value_counts()
print(species_total)

# Where are most bites occurring?
zip_location = df['victim_zip'].value_counts()
print(zip_location.head(10))

PIT BULL           1078
GERM SHEPHERD       323
LABRADOR RETRIV     248
BOXER               181
CHICHAUHUA          164
Name: BreedIDDesc, dtype: int64
['CAT' 'DOG' 'BAT' 'RACCOON' 'OTHER' None 'HORSE' 'RABBIT' 'SKUNK'
 'FERRET']
DOG        6883
CAT        1529
BAT          76
RACCOON      21
OTHER         8
HORSE         5
FERRET        4
RABBIT        3
SKUNK         1
Name: SpeciesIDDesc, dtype: int64
40272    371
40291    354
40216    354
40215    342
40214    335
40299    320
40229    294
40219    278
40218    246
40245    234
Name: victim_zip, dtype: int64


+ Fewer horse bites than one may assume for Kentucky. 
+ With seventy-six reported bat bites, I now question some of the information that documentarian David Attenborough has mentioned, seemingly contradicting how he claims bats behave. 
+ I also wonder if our one unfortunate skunk-bite victim knows how unique their situation really was.
+ Visualizations for these can be found on my [Tableau Dashboard](https://public.tableau.com/views/AnimalBitesandLunacy/dashboard_animal_bites_and_lunacy?:language=en-US&:display_count=n&:origin=viz_share_link)

Here we look at the total counts of each individual phase.

In [95]:
# How many total bites occurred during each moon phase?
phases_total = df["phase"].value_counts()
print(phases_total)

waning gibbous     1352
waning crescent    1300
waxing gibbous     1295
waxing crescent    1279
last quarter        883
first quarter       863
full                832
new                 827
Name: phase, dtype: int64


Full moons appear to be on the low end of total reported bites.

## Correlation

To find any correlation between moon phases and bites we will also need to include dates in which no bites were reported. For this we will be better off making another SQL query to tailor a separate dataframe to our parameters. We'll start with **ALL** moon phases and dates, and join bite data to include the necessary data.

In [96]:
# SQL request for moon data for all dates in this timeframe
moon_corr_df = pd.read_sql("""SELECT 
                            date, 
                            phase 
                        FROM 
                            moon m 
                        WHERE 
                            m.date 
                        BETWEEN 
                            '2009-10-29' and '2017-09-08' 
                        ORDER BY 
                            date;""", conn)

# DEBUG make a function out of this?
# again convert to datetime, split on date, convert to datetime
moon_corr_df['date'] = pd.to_datetime(moon_corr_df['date'])
moon_corr_df['date'] = moon_corr_df['date'].dt.date
moon_corr_df['date'] = pd.to_datetime(moon_corr_df['date'])

We'll now make a secondary dataframe off of our bite_df dataframe, but without some unnecessary columns.

In [97]:
# We only need dates and NaTs for creating booleans
bite_corr_df = bite_df['bite_date'].copy()
print(bite_corr_df.head())

0   2009-10-29
1   2009-12-02
2   2009-12-11
3   2009-12-21
4   2009-12-24
Name: bite_date, dtype: datetime64[ns]


This merge will create a dataframe that has ALL dates from our selected range, not just the dates where bites occured.

In [98]:
# Merge on date to keep dates not included in bite_date
corr_df = pd.merge(moon_corr_df, bite_corr_df, how='left', left_on='date', right_on='bite_date')
print(corr_df.head())

        date           phase  bite_date
0 2009-10-29  waxing gibbous 2009-10-29
1 2009-10-30  waxing gibbous        NaT
2 2009-10-31  waxing gibbous        NaT
3 2009-11-01            full        NaT
4 2009-11-02            full        NaT


We'll create a new column that serves as a boolean for whether or not a bites occured on a given date.

In [99]:
# Create a new column 'new_col' with 1's and 0's
corr_df['incident'] = corr_df['bite_date'].apply(lambda x: 1 if not pd.isna(x) else 0)

print(corr_df)

           date           phase  bite_date  incident
0    2009-10-29  waxing gibbous 2009-10-29         1
1    2009-10-30  waxing gibbous        NaT         0
2    2009-10-31  waxing gibbous        NaT         0
3    2009-11-01            full        NaT         0
4    2009-11-02            full        NaT         0
...         ...             ...        ...       ...
8850 2017-09-06            full 2017-09-06         1
8851 2017-09-07            full 2017-09-07         1
8852 2017-09-07            full 2017-09-07         1
8853 2017-09-07            full 2017-09-07         1
8854 2017-09-07            full 2017-09-07         1

[8855 rows x 4 columns]


First we'll create a dictionary which maps each moon phase to a unique number. After which a new column is created to store the corresponding value.

In [100]:
# Create a dictionary mapping each phase string to a numerical value
phase_dict = {'new': 1, 'waxing crescent': 2, 'first quarter': 3, 
              'waxing gibbous': 4, 'full': 5, 'waning gibbous': 6, 
              'last quarter': 7, 'waning crescent': 8}


# Map the 'phase' column to the numerical values using the dictionary
corr_df['phase_map'] = corr_df['phase'].map(phase_dict)

Next we will create yet another, more specific dataframe that has only the columns which will not posess null values. Pandas needs numbers for calculating correlation, and any other data types will not allow the operation to function.

In [101]:
# Select only the columns used for the correlation calculation
corr_cols = ['incident', 'phase_map']

# Lets go ahead and make this its own special purpose dataframe
special_corr_df = corr_df[corr_cols]

# Calculate the correlation matrix between the two columns
correlation = special_corr_df.corr()

# Print the correlation coefficient between the two columns
print('Correlation coefficient:', correlation['incident']['phase_map'])


Correlation coefficient: -0.005128849435105016


A correlation coefficient of -0.005129 is **very close** to zero. This suggests there is little to no linear relationship between the moon phase and the incidence of animal bites reported in Louisville. The moon phase and the incidence of bites are essentially independent of each other. This coefficient suggests that the moon phase is **not** a strong predictor of animal bites, and that other factors may be more important in determining when bites will occur.

In summation, we conclude with a brief heat map visualization.

In [102]:
# Correlation visualization
correlation.style.background_gradient(cmap="Reds")


,incident,phase_map
incident,1.000000,-0.005129
phase_map,-0.005129,1.000000


In [103]:
# Be sure to close the door behind you
conn.close()